In [1]:
import osmnx as ox
import pandas as pd

In [2]:
# Example: get driving network for a city
# G = ox.graph_from_place("Somerville, Massachusetts, USA", network_type="bike")
G = ox.graph_from_place("New York City, New York, USA", network_type="bike")

In [3]:
# Convert to GeoDataFrames (nodes + edges)
nodes, edges = ox.graph_to_gdfs(G)

In [4]:
edges.sample(10)

,,,osmid,highway,maxspeed,name,oneway,reversed,length,geometry,lanes,ref,service,access,bridge,width,tunnel,junction,est_width
u,v,key,,,,,,,,,,,,,,,,,
2547411482,42469274,0,247886811,tertiary,NaN,Sutter Avenue,False,False,75.259780,"LINESTRING (-73.90166 40.66841, -73.90159 40.6...",2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42878072,5488162694,0,420304087,primary,25 mph,Merrick Boulevard,True,False,29.302055,"LINESTRING (-73.78086 40.69387, -73.78105 40.6...",2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8130252108,7863107944,0,"[873098987, 842882358]",service,NaN,NaN,False,False,89.166678,"LINESTRING (-73.81003 40.65695, -73.80987 40.6...",NaN,NaN,parking_aisle,NaN,NaN,NaN,NaN,NaN,NaN
42466085,42505239,0,5679039,residential,NaN,90th Street,True,False,236.736723,"LINESTRING (-74.0322 40.62069, -74.03232 40.62...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42469045,42488006,0,545471779,secondary,NaN,86th Street,False,False,82.884267,"LINESTRING (-73.99381 40.60194, -73.99371 40.6...",2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7768645412,42720163,0,141119721,residential,NaN,West 236th Street,False,False,41.284336,"LINESTRING (-73.90654 40.88574, -73.90685 40.8...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42858815,42922723,0,5710067,residential,NaN,157th Street,False,True,204.559478,"LINESTRING (-73.7798 40.67159, -73.77985 40.67...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10717967553,42514037,0,1149556527,primary,NaN,Kings Highway,False,True,39.244784,"LINESTRING (-73.92471 40.64606, -73.92478 40.6...",4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3843180751,277482105,0,194933380,unclassified,NaN,Sunken Meadow Loop,False,True,45.907608,"LINESTRING (-73.91804 40.7966, -73.91819 40.79...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
highways = edges.reset_index()['highway']

In [24]:
unique = set()
for _, v in highways.items():
    if isinstance(v, list):
        unique.update(v)
    else:
        unique.add(v)
unique

{'busway',
 'cycleway',
 'living_street',
 'path',
 'pedestrian',
 'primary',
 'primary_link',
 'residential',
 'secondary',
 'secondary_link',
 'service',
 'tertiary',
 'trunk',
 'trunk_link',
 'unclassified'}

In [ ]:
import numpy as np
import re

In [4]:
edges[['highway', 'name', 'maxspeed']].sample(10)

,,,highway,name,maxspeed
u,v,key,,,
7983674950,71916827,0,residential,Belmont Street,20 mph
11777326029,11777326029,1,service,NaN,NaN
61172626,61174071,0,trunk,Fellsway,NaN
1949661741,1155703191,0,residential,Latin Way,NaN
8334781996,8334781999,0,service,NaN,NaN
61170625,61170638,0,secondary,Summer Street,20 mph
61170914,71917815,0,secondary,Summer Street,20 mph
71914380,71927580,0,residential,Edgar Avenue,20 mph
71947029,71921397,0,secondary,Grand Union Boulevard,20 mph


In [7]:
def extract_maxspeed(x):
    # Handle NaN early
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return np.nan

    # Handle lists and numpy arrays uniformly
    if isinstance(x, (list, np.ndarray)):
        speeds = [extract_maxspeed(i) for i in x]
        # filter out NaNs
        speeds = [s for s in speeds if not pd.isna(s)]
        return max(speeds) if speeds else np.nan

    # Convert to string and extract digits
    x_str = str(x)
    nums = re.findall(r'\d+', x_str)

    return int(max(nums, key=int)) if nums else np.nan

In [8]:
edges['maxspeed_int'] = edges['maxspeed'].apply(extract_maxspeed)

In [9]:
edges[['highway', 'name', 'maxspeed', 'maxspeed_int']].sample(10)

,,,highway,name,maxspeed,maxspeed_int
u,v,key,,,,
71951018,71928807,0,residential,Stickney Avenue,20 mph,20.0
61173051,71938041,0,residential,Dickson Street,NaN,NaN
71916774,71953040,0,residential,Victoria Street,NaN,NaN
71954205,945798791,0,primary,Somerville Avenue,25 mph,25.0
61182360,61178893,0,secondary,Medford Street,25 mph,25.0
61177298,11395248465,0,residential,Central Road,25 mph,25.0
604063583,71936370,0,residential,Jaques Street,20 mph,20.0
71920826,71947231,0,secondary,Broadway,25 mph,25.0
71919864,71949322,0,residential,Century Street,20 mph,20.0


In [11]:
edges.to_file("../data/somerville_roads.gpkg", layer="roads", driver="GPKG")

## Bike

In [18]:
# Example: get driving network for a city
G = ox.graph_from_place("Somerville, Massachusetts, USA", network_type="bike")

In [19]:
# Convert to GeoDataFrames (nodes + edges)
nodes, edges = ox.graph_to_gdfs(G)

In [17]:
edges.to_file("../data/somerville_bike.gpkg", layer="roads", driver="GPKG")